In [ ]:
import os
from datetime import datetime, date, timedelta
import pandas as pd 


In [ ]:
folder_path = 'D:/Offline Data/Original Data/Nifty_1_Second_ICICI'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
sorted_files = sorted(csv_files, key=lambda x: datetime.strptime(x.split('_')[0] + '_' + x.split('_')[1] + '_' + x.split('_')[2], '%Y_%m_%d'))

for i in range(0, len(sorted_files)):
    print(sorted_files[i])

    file_path = os.path.join(folder_path, sorted_files[i])
    df = pd.read_csv(file_path)

    # ----------------------------------- Only for row file -----------------------------------
    
    def create_symbol(row):
        expiry_date = datetime.strptime(row['expiry_date'], '%d-%b-%Y')
        date_str = f"{expiry_date.day:02d}"
        month_str = expiry_date.strftime('%b').upper()
        year_str = str(expiry_date.year)[-2:]
        
        strike_str = str(int(row['strike_price']))
        right_str = 'CE' if row['right'].lower() == 'call' else 'PE'
        
        return f"{row['stock_code']}50{date_str}{month_str}{year_str}{strike_str}{right_str}"

    df['symbol'] = df.apply(create_symbol, axis=1)
    df = df[['datetime', 'open', 'high', 'low', 'close', 'symbol', 'open_interest', 'volume']]

    # -------------------------- convert data into required timeframe -------------------------

    df = df.copy()

    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values(['datetime', 'symbol'])

    df_5sec = []
    for symbol in df['symbol'].unique():
        symbol_data = df[df['symbol'] == symbol].set_index('datetime')
        resampled = symbol_data.resample('3T').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'open_interest': 'last',
            'volume': 'sum'
        }).dropna()
        resampled['symbol'] = symbol
        df_5sec.append(resampled)

    df_5sec = pd.concat(df_5sec).reset_index()
    df = df_5sec[['datetime', 'symbol', 'open', 'high', 'low', 'close', 'open_interest', 'volume']]

    save_folder_path = f'D:/Offline Data/Processed Data/Nifty_3_Minute_ICICI'
    save_file_path = os.path.join(save_folder_path, sorted_files[i].replace('1second', '3_Minute'))
    df.to_csv(save_file_path, index=False)
